In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_excel('/home/dariodg/Desktop/free/Galpao/ComissionApp/docs/comissabril2019.XLS')

In [3]:
df.columns

Index(['turma', 'titulo', 'vencimento', 'sacado', 'recebido', 'credito',
       'descritivo'],
      dtype='object')

## Turmas - Adicionando "sem turma" aos valores vazios

In [4]:
turmas = df['turma']

In [5]:
turmas.describe()

count                          218
unique                          35
top       2019 AÉREOS 2x - TURMA 3
freq                            18
Name: turma, dtype: object

In [6]:
print(type(turmas[0]) == float) # Valor sem turma é considerado false, tipo float

True


In [7]:
def fill_sem_turma(value):
    if type(value) == float:
        return 'sem_turma'
    return value

In [8]:
turma2 = turmas.apply(fill_sem_turma)

In [9]:
turma2.head()

0                      sem_turma
1    2019 AC - ADOLESC - TURMA 1
2    2019 AC - ADOLESC - TURMA 1
3    2019 AC - ADOLESC - TURMA 1
4    2019 AC - ADOLESC - TURMA 1
Name: turma, dtype: object

In [10]:
tuple(df.head(0))

('turma',
 'titulo',
 'vencimento',
 'sacado',
 'recebido',
 'credito',
 'descritivo')

In [11]:
type(df.head(0))

pandas.core.frame.DataFrame

In [12]:
descritivo = df['descritivo']

In [13]:
descritivo.head()

0    parcela de anuidade - 3ª/12             R$ 435...
1    TAXA PROD - 3ª/12         R$ 435,00\nARTE CIRC...
2    ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00\nTAXA PROD...
3    TX ADESÃO - 1ª/1          R$ 220,00\nTAXA PROD...
4    ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00\nTAXA PROD...
Name: descritivo, dtype: object

In [14]:
def strip_n_split(value):
    value = value.strip()
    value = value.split('\n')
    return value

In [22]:
descritivo2 = descritivo.apply(strip_n_split)

In [23]:
descritivo2[0]

['parcela de anuidade - 3ª/12             R$ 435,00',
 'Valor de produção - AÉR INI PAL - 3ª/12 R$  16,66']

In [24]:
descritivo2

0      [parcela de anuidade - 3ª/12             R$ 43...
1      [TAXA PROD - 3ª/12         R$ 435,00, ARTE CIR...
2      [ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00, TAXA PRO...
3      [TX ADESÃO - 1ª/1          R$ 220,00, TAXA PRO...
4      [ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00, TAXA PRO...
                             ...                        
214    [ARTE CIRC  ADOL 1 - 8ª/12 R$ 207,50, Taxa pro...
215    [ARTE CIRC  ADOL 1 - 9ª/12 R$ 26,23, Taxa prod...
216    [Multas a receber        R$  37,00, AÉR 2X - A...
217    [Multas a receber        R$  40,00, BRINCADEIR...
218                 [INICIAÇÃO CRI 1 - 11ª/12 R$ 275,00]
Name: descritivo, Length: 219, dtype: object

In [25]:
goal = ['parcela de anuidade - 3ª/12             R$ 435,00', 'Valor de produção - AÉR INI PAL - 3ª/12 R$  16,66']

In [26]:
descritivo2[0] == goal

True

In [30]:
descritivo2[3]

['TX ADESÃO - 1ª/1          R$ 220,00',
 'TAXA PROD - 1ª/10         R$  20,00',
 'ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00']

In [31]:
df['sacado'][3]

'Giulia Marandino Diaz'